In [1]:
from trimesh import *
from swf import *
from optimal import *
from utils import *
import numpy as np
import soundfile as sf
from scipy import spatial

def random_three_vector():
    """
    Generates a random 3D unit vector (direction) with a uniform spherical distribution
    Algo from http://stackoverflow.com/questions/5408276/python-uniform-spherical-distribution
    :return:
    """
    phi = np.random.uniform(0,np.pi*2)
    costheta = np.random.uniform(-1,1)

    theta = np.arccos( costheta )
    x = np.sin( theta) * np.cos( phi )
    y = np.sin( theta) * np.sin( phi )
    z = np.cos( theta )
    return (x,y,z)

In [2]:
layout704=np.array([[1,np.pi/6,np.pi/2],[1,-np.pi/6,np.pi/2],[1,0,np.pi/2],[1,np.pi/2,np.pi/2],[1,-np.pi/2,np.pi/2],[1,3*np.pi/4,np.pi/2],[1,-3*np.pi/4,np.pi/2],[1,np.pi/4,np.pi/4],[1,-np.pi/4,np.pi/4],[1,3*np.pi/4,np.pi/4],[1,-3*np.pi/4,np.pi/4]])
vertices704 = np.apply_along_axis(lambda x: toCartesian(x),1,layout704)
faces704 = np.array([[6,4,10],[10,4,8],[8,4,1],[8,1,2],[8,7,2],[7,2,0],[7,0,3],[7,3,9],[9,3,5],[10,7,9],[10,8,7],[10,6,5],[10,9,5]])

model = OptimalSWF(vertices704,faces704).model

In [54]:
loc = tree.query(np.array([-1,0,0.5]))[1]
loc

251

In [55]:
data, samplerate = sf.read('bossa.wav')

fine = np.zeros((model.meshes[-1].vertices.shape[0],data.shape[0]))
tree = spatial.KDTree(model.meshes[-1].vertices)
#loc = tree.query([random_three_vector()])[1][0]
onehot = np.zeros((model.meshes[-1].vertices.shape[0],1))
onehot[:11] = -1
onehot[loc]=1
fine[loc] = data
coarse = model.encode(fine)

In [56]:
weights3D(model.meshes[-1],onehot,'point source location')

In [57]:
coarse = np.insert(coarse, 3, 0, axis=0) #add empty subwoofer channel for reproduction in the studio
coarse.shape

(12, 256000)

In [58]:
sf.write('encoded.wav',coarse.T,samplerate)